In [ ]:
!pip install konlpy

In [ ]:
!pip install -U pyLDAvis

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('다이소_유튜브_제목.csv')
df.columns = ['0', '제목']
df.head()

In [ ]:
df.drop("0",axis=1,inplace=True)

In [ ]:
df

In [ ]:
from konlpy.tag import Okt
import re

In [ ]:
def tokenize_text(text):
    text = re.sub(r"[^ㄱ-ㅣ가-힣\s]","",text)
    okt = Okt()
    okt_morphs = okt.pos(text)

    words = []
    for word,pos in okt_morphs: 
        if pos == 'Adjective' or pos=='Verb' or pos=='Noun':
            words.append(word)

    word_str =  ' '.join(words)  
    return word_str

In [ ]:
from tqdm import tqdm
token_list = []
for temp in tqdm(df['제목']):
    token_list.append(tokenize_text(temp))
token_list

In [ ]:
drop_corpus = []

for index in range(len(token_list)):
    corpus = token_list[index]
    if len(set(corpus.split())) < 3:
        drop_corpus.append(corpus)

for corpus in drop_corpus:
    token_list.remove(corpus)

token_list

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
count_vec = CountVectorizer(max_df=0.2,max_features=1000,min_df=3,ngram_range=(1,2))
feat_vect = count_vec.fit_transform(token_list)
print(feat_vect.shape)
print(count_vec.vocabulary_)

In [ ]:
feature_names = count_vec.get_feature_names_out()

In [ ]:
lda = LatentDirichletAllocation(n_components=5,max_iter=20)
lda.fit(feat_vect)

In [ ]:
lda.components_

In [ ]:
def display_topics(model,feature_names,num_top_words):
    for topic_index, topic in enumerate(model.components_):
        print('토픽',topic_index)
        topic_word_indexes = topic.argsort()[::-1]
        top_index = topic_word_indexes[:num_top_words]
       
        f_name_list = []
        for temp in top_index:
            f_name_list.append(feature_names[temp])

        feature_concat = ' '.join(f_name_list)
        print(feature_concat)
    

In [ ]:
display_topics(lda,feature_names,15)

In [ ]:
import pyLDAvis.lda_model
pyLDAvis.enable_notebook()
vis = pyLDAvis.lda_model.prepare(lda,feat_vect,count_vec)
pyLDAvis.display(vis)

In [ ]:
sent_topic = lda.transform(feat_vect)
print(sent_topic[0])

In [ ]:
doc_per_topic_list = []
for n in range(sent_topic.shape[0]):
    topic_most_pr = sent_topic[n].argmax()
    topic_pr = sent_topic[n].max()
    doc_per_topic_list.append([n,topic_most_pr,topic_pr])

doc_topic_df = pd.DataFrame(doc_per_topic_list,columns=['no','토픽번호','확률'])
doc_topic_df



In [ ]:
for topic in range(len(doc_topic_df['토픽번호'].unique())):
    print('토픽',topic)
    top_topic = doc_topic_df[ doc_topic_df['토픽번호']==topic].sort_values(by='확률',ascending=False)
    print(df['제목'].iloc[ top_topic['no'].iloc[0]])
    print(df['제목'].iloc[ top_topic['no'].iloc[1]])
    print(df['제목'].iloc[ top_topic['no'].iloc[2]])

# 분석 결과

**1**  
정리, 가지, 품절, 게시 살림, 출시, 구매, 주방, 즉시, 입고, 입고 즉시, 즉시 품절, 급상승, 매출, 다이소 꿀템, 살림 개월, 매출 급상승, 신상, 하세요, 다이소 신상, 꿀템 가지, 꿀팁, 베스트 셀러, 구매 하세요, 수납, 출시 매출, 다이소 출시, 만들기, 추천 가지, 추천 다이소

**2**   
하는, 리뷰, 후호, 가지, 게시 리뷰, 캠핑, 용품, 리뷰 하는, 가성, 하는 호랭, 호랭, 모르면, 모르면 후호, 베품, 호랭 개월, 가기, 후회 다이소, 사세요, 다이소 가기, 영상, 추천 게시, 추천 가지, 다이소 꿀템, 생활, 이건, 가면, 정리, 야할, 꿀템 게시, 캠핑 용품

**3**    
구매, 보이, 사세요, 가지, 하기, 게시 짤컷, 짤것, 짤것 개월, 다이소 꿀템, 게시 살림, 당장, 게시 개월, 쇼핑, 베스트, 가자 게시, 화장품, 셀러, 베스트 셀러, 다이소 보이, 직원, 품절, 가성, 무조건, 진짜, 하세요, 구매 하세요, 하기 게시, 고민, 무조건 사세요

**4**     
천원, 상품, 신상, 짜리, 리뷰, 게시 다이소, 다이소 상품, 다이소 신상, 게시 살림, 시간, 다이소 천원, 후회, 리뷰 게시, 하는, 다이소 다이소, 추천 게시, 다이소 꿀템, 인기, 추천 다이소, 주차, 주차 다이소, 출시, 사야, 뷰티, 남자, 로그, 지금, 일상, 하는 다이소, 다이소 개월

**5**    
다이소 꿀템, 꿀템 다이소, 추천 다이소, 다이소 다이소, 신상, 가지, 다이소 신상, 다이소 살림, 살림 다이소, 게시 살림, 직원, 몰래, 가지 다이소, 포장, 산다, 신상 다이소, 활용, 추천 가지, 이편, 이편 한살림, 한살림, 산다 다이소, 생활용품, 수납, 차박, 생활, 직원 몰래, 품절, 게시 개월


#### **결론**

토픽 모두 공통적으로 '추천','꿀템','꿀팁','신상'등의 키워드를 볼 수 있었으며, 유튜브 다이소 영상의 대부분이이 다이소 물건을 추천하거나, 활용 방법등을 소개하는 내용임을을 알 수 있었다.